In [ ]:
import json
import ruamel.yaml
import os
import requests
import time
import base64
from PIL import Image
from io import BytesIO
import aria2p
import concurrent.futures
import urllib

yaml = ruamel.yaml.YAML()

In [ ]:
def get_csrf_token():
    url = "https://buct.smartclass.cn/wechat.json?r=2021-01-29"
    response = requests.get(url)
    wechat_json = response.content.decode('utf-8-sig') 
    csrkKey = json.loads(wechat_json)["csrkKey"]

    t = int(time.time() * 1000)
    token = ""
    j = 1000000000000
    while j != 0:
        k = (t - t % j) // j
        token += csrkKey[k]
        t -= k * j
        j = j // 10
    return token

def check_login(session):
    url = "https://buct.smartclass.cn/Permission/DataBoardPageList?csrkToken=" + get_csrf_token()
    response = session.get(url, allow_redirects=False)
    if response.status_code == 200:
        return True
    else:
        print("Login failed using saved cookies.")
        print(response.status_code)
        return False

def load_cookies():
    cookies = {
        "PowerCreatorCMSAuthenticationID": PowerCreatorCMSAuthenticationID,
        "platform_token": platform_token
    }
    return cookies

def save_cookies_to_yaml(session):
    cookies = session.cookies.get_dict()
    with open('config.yaml', 'r', encoding='utf-8') as f:
        config = yaml.load(f)

    config["authorization"]["PowerCreatorCMSAuthenticationID"] = cookies["PowerCreatorCMSAuthenticationID"]
    config["authorization"]["platform_token"] = cookies["platform_token"]

    with open('config.yaml', 'w', encoding='utf-8') as f:
        yaml.dump(config, f)

def login():
    session = requests.Session()
    cookies = load_cookies()
    if cookies:
        session.cookies.update(cookies)
        if check_login(session):
            print("Login successful using saved cookies.")
            return session

    url = "https://buct.smartclass.cn/Login.aspx"
    response = session.get(url)
    verfycodejson = response.text
    random = str(int(time.time() * 1000000))
    random = str(float(random) * 0.0000001)
    url = f"https://buct.smartclass.cn/home/VerifyImage2?{random}"
    response = session.get(url)
    verfycodejson = response.text
    verfyimg = json.loads(verfycodejson)["Value"]["img"]
    verfycode = json.loads(verfycodejson)["Value"]["Code"]
    verfyimg = "data:image/png;base64," + verfyimg
    # 将 base64 编码的图片数据转换为二进制数据
    img_data = base64.b64decode(verfyimg.split(",")[1])

    # 使用 PIL 库打开图片
    img = Image.open(BytesIO(img_data))

    # 显示验证码图片
    img.show()

    # 获取用户输入的验证码
    code = input("Please enter the verification code: ")

    # 关闭图片窗口
    img.close()
    url = "https://buct.smartclass.cn/Home/Login"
    data = {
        "UserName": username,
        "PassWord": password,
        "isRember": "true",
        "VerifyCode": code,
        "VerifyCodeId": verfycode,
        "redirectUri": ""
    }
    response = session.post(url, data=data)
    print(response.text)
    # 检查登录是否成功
    response_data = json.loads(response.text)
    if response_data.get("Success"):
        print("Login successful.")
        save_cookies_to_yaml(session)
        return session
    else:
        print("Login failed.")
        return login()


In [ ]:
# 读取配置文件
with open('config.yaml', 'r', encoding='utf-8') as f:
    config = yaml.load(f)
# 从配置文件中获取授权信息
authorization = config.get('authorization', {})
PowerCreatorCMSAuthenticationID = authorization.get('PowerCreatorCMSAuthenticationID')
platform_token = authorization.get('platform_token')
username = authorization.get('username')
password = authorization.get('password')

# 从配置文件中获取Aria2设置
downloader = config.get('downloader', {})
aria2_host = downloader.get('aria2_host', 'http://localhost')
aria2_port = downloader.get('aria2_port', 6800)
aria2_secret = downloader.get('aria2_secret', '')
aria2 = aria2p.API(aria2p.Client(host=aria2_host, port=aria2_port, secret=aria2_secret))

# 从配置文件中获取base_dir
base_dir = downloader.get('base_dir')

# 从配置文件中获取预览相关信息
priview = config.get('priview', {})
website = priview.get('website_url')
base_urls = {
    "in": priview.get('base_urls_in'),
    "out": priview.get('base_urls_out')
}

# 如果PowerCreatorCMSAuthenticationID和platform_token都设置，则不需要账号密码
if PowerCreatorCMSAuthenticationID and platform_token:
    pass
# 如果有账号密码，没有token和id，也可以
elif username and password:
    pass
# 如果都没有，让用户选择输入哪对
else:
    print("请选择输入方式：")
    print("1. 输入PowerCreatorCMSAuthenticationID和platform_token")
    print("2. 输入账号和密码")
    choice = input("请输入选择（1或2）: ")
    
    if choice == "1":
        print("提示：PowerCreatorCMSAuthenticationID和platform_token可以用F12查看，但是临时的，会过期。")
        PowerCreatorCMSAuthenticationID = input("请输入PowerCreatorCMSAuthenticationID: ")
        config['authorization']['PowerCreatorCMSAuthenticationID'] = PowerCreatorCMSAuthenticationID
        platform_token = input("请输入platform_token: ")
        config['authorization']['platform_token'] = platform_token
    elif choice == "2":
        print("提示：账号和密码需要通过抓包浏览器登录的请求获取。")
        username = input("请输入加密后的用户名: ")
        config['authorization']['username'] = username
        password = input("请输入加密后的密码: ")
        config['authorization']['password'] = password
    else:
        print("无效的选择，请重新运行程序。")
        exit()

    with open('config.yaml', 'w', encoding='utf-8') as f:
        yaml.dump(config, f)

# 登录
session = login()

In [ ]:
def get_course_list():
    token = get_csrf_token()
    url = f"https://buct.smartclass.cn/WebApi/V1/video/GetCourseLists?csrkToken={token}&schoolId=289&order=&sort=StartTime&Source=&IsDeleted=false&IsProcessSuccess=&IsProcessing=&IsNeedLive=&CollectorID=&ViewerID=&IsPublic=&IsPublish=true&stopDate=&startDate=&ClassroomID=&CourseID=&organizationID=&StudentID=13248246&TeacherID=&NodeID=&titleKey=&pageNumber=1&pageSize=100&GradeID=&attribute=&type=0&ActivityID="
    response = session.get(url)
    course_list = json.loads(response.text)
    return course_list

def get_course_videos(course_info):
    token = get_csrf_token()
    # get needed information
    teacher_id = course_info["TeacherID"]
    course_id = course_info["CourseID"]
    user_id = course_info["UserID"]
    class_id = course_info["ClassID"]
    # get course video
    url = "https://buct.smartclass.cn/WebApi/V1/video/GetCourseVideos"
    data = {
        "ClassRoomId": "",
        "csrkToken": token,
        "pageSize": 100,
        "pageNumber": 1,
        "Sort": "StartTime",
        "order": 1,
        "IsDeleted": False,
        "ChapterTreeIDs": "",
        "TeacherID": teacher_id,
        "courseID": course_id,
        "userID": user_id,
        "classId": class_id,
        "type": 0,
        "IsPublish": True
    }
    response = session.post(url, data=data)
    video_list = json.loads(response.text)
    return video_list

def get_valid_url(base_url, replacements):
    """
    根据给定的 base_url 和替换规则,获取有效的视频 URL。

    Args:
        base_url (str): 基础 URL。
        replacements (list): 替换规则列表,每个元素都是一个元组,包含要替换的字符串和替换后的字符串。

    Returns:
        str: 有效的视频 URL,如果没有找到有效的 URL,则返回 None。
    """
    session = requests.Session()
    valid_urls = []

    for replacement in replacements:
        url_0 = base_url.replace(*replacement)
        url_1 = base_url.replace(*replacement[::-1])

        response_0 = session.head(url_0)
        response_1 = session.head(url_1)

        if (
            response_0.status_code == 200
            and response_0.headers.get("Content-Type") == "video/mp4"
            and "Content-Length" in response_0.headers
        ):
            valid_urls.append((url_0, int(response_0.headers["Content-Length"])))

        if (
            response_1.status_code == 200
            and response_1.headers.get("Content-Type") == "video/mp4"
            and "Content-Length" in response_1.headers
        ):
            valid_urls.append((url_1, int(response_1.headers["Content-Length"])))

    if valid_urls:
        return min(valid_urls, key=lambda x: x[1])[0]

    return None

def download_video(video_info):
    pic_url = video_info["PicUrl"]
    title = video_info["Title"]
    start_time = video_info["StartTime"]
    start_time = start_time.replace(":", "-").replace(" ", "_")[:-3][2:]
    # 实际上的下载路径是title第一个空格前的部分-第二个空格前的部分
    download_path = os.path.join(base_dir, title.split(" ")[0] + "-" + title.split(" ")[1])
    if not os.path.exists(download_path):
        os.makedirs(download_path)
   
    vga_file = os.path.join(download_path, start_time + ".VGA.mp4")
    video1_file = os.path.join(download_path, start_time + ".Video1.mp4")
    vga_aria2_file = os.path.join(download_path, start_time + ".VGA.mp4.aria2")
    video1_aria2_file = os.path.join(download_path, start_time + ".Video1.mp4.aria2")
    
    if (os.path.exists(vga_aria2_file)) or (os.path.exists(video1_aria2_file)):
        print(f"{title} already downloaded but not finished.")
        os.remove(vga_file)
        os.remove(vga_aria2_file)
        os.remove(video1_file)
        os.remove(video1_aria2_file)
    if os.path.exists(os.path.join(download_path, start_time + ".VGA.mp4")) and os.path.exists(os.path.join(download_path, start_time + ".Video1.mp4")):
        print(f"{title} already downloaded.")
        return
    vga_url = get_valid_url(pic_url.replace("000.jpg", "VGA.mp4"), [("/1/", "/0/")])
    video1_url = get_valid_url(pic_url.replace("000.jpg", "video1.mp4"), [("/1/", "/0/")])
    if vga_url:
        # download video
        print(f"Downloading {title}...")
        aria2.add_uris([vga_url], options={"dir": download_path, "out": start_time + ".VGA.mp4"})
    if video1_url:
        # download video
        print(f"Downloading {title}...")
        aria2.add_uris([video1_url], options={"dir": download_path, "out": start_time + ".Video1.mp4"})

def write_watch_list(video_infos):
    title = video_infos[0]["Title"]
    path = title.split(" ")[0] + "-" + title.split(" ")[1]

    # if path not exists, create it
    if not os.path.exists(os.path.join(base_dir, path)):
        os.makedirs(os.path.join(base_dir, path))
    
    def format_url(video_type, in_out, start_time):
        video_url = urllib.parse.quote(f"{base_urls[in_out]}/{path}/{start_time}.{video_type}.mp4")
        subtitle_url = urllib.parse.quote(f"{base_urls[in_out]}/{path}/{start_time}.Video1.json")
        return f"{website}?video={video_url}&subtitle={subtitle_url}"

    def write_table_row(f, start_time, file_type):
        video1_in_url = format_url("Video1", "in", start_time)
        vga_in_url = format_url("VGA", "in", start_time)
        video1_out_url = format_url("Video1", "out", start_time)
        vga_out_url = format_url("VGA", "out", start_time)
        
        if file_type == "md":
            f.write(f"| {start_time} | [Video1]({video1_in_url}) | [VGA]({vga_in_url}) | [Video1]({video1_out_url}) | [VGA]({vga_out_url}) |\n")
        else:
            f.write(f"<tr><td>{start_time}</td><td><a href='{video1_in_url}' target='_blank'>Video1</a></td><td><a href='{vga_in_url}' target='_blank'>VGA</a></td><td><a href='{video1_out_url}' target='_blank'>Video1</a></td><td><a href='{vga_out_url}' target='_blank'>VGA</a></td></tr>\n")

    with open(os.path.join(base_dir, path, path + ".md"), "w", encoding="utf-8") as f:
        f.write("| StartTime | Video1 | VGA | Video1 | VGA |\n")
        f.write("| --- | --- | --- | --- | --- |\n")
        for video_info in video_infos:
            start_time = video_info["StartTime"].replace(":", "-").replace(" ", "_")[:-3][2:]
            write_table_row(f, start_time, "md")

    with open(os.path.join(base_dir, path, path + ".html"), "w", encoding="utf-8") as f:
        f.write("<html><head><title>Watch List</title></head><body>")
        f.write("<table>")
        f.write("<tr><th>StartTime</th><th>Video1</th><th>VGA</th><th>Video1</th><th>VGA</th></tr>")
        for video_info in video_infos:
            start_time = video_info["StartTime"].replace(":", "-").replace(" ", "_")[:-3][2:]
            write_table_row(f, start_time, "html")
        f.write("</table>")
        f.write("</body></html>")


In [ ]:
course_list = get_course_list()
futures = []
for course_info in course_list["Value"]["Data"]:
    video_list = get_course_videos(course_info)
    write_watch_list(video_list["Value"]["Data"])

with concurrent.futures.ThreadPoolExecutor(max_workers=16) as executor:
    for course_info in course_list["Value"]["Data"]:
        video_list = get_course_videos(course_info)
        write_watch_list(video_list["Value"]["Data"])
        for video_info in video_list["Value"]["Data"]:
            future = executor.submit(download_video, video_info)
            futures.append(future)